In [ ]:
import pickle
import os
os.environ["PRIVBAYES_BIN"] = "./ydnpd/synthesis/privbayes"

import ray
from IPython.display import display, Markdown

import ydnpd

In [ ]:
ray.init()

In [ ]:
print(f"Task sizes: {[task.size() for task in ydnpd.span_hparam_tasks()]}")

## Execute Tasks

In [ ]:
flatten_hparam_tasks_results = ray.get(ydnpd.span_hparam_ray_tasks())

In [ ]:
pickle.dump(flatten_hparam_tasks_results, open("flatten_hparam_tasks_results.pkl", "wb"))

In [ ]:
hparam_tasks_results = ydnpd.collect_hparam_runs(flatten_hparam_tasks_results)

## Task: Hyperparameter Search

In [ ]:
for synth_name, (hparam_task, hparam_results) in hparam_tasks_results.items():

    for dev_name, test_name in ydnpd.config.EXPERIMENTS:
        
        display(Markdown(f"## {synth_name}: {dev_name} × {test_name}"))

        hparam_task.plot(hparam_results,
                         dev_name=dev_name,
                         test_name=test_name,
                         metric="marginals_up_to_3_max_abs_diff_errors")

## Task: Privacy-Utility Trade-off

In [ ]:
for synth_name, (hparam_task, hparam_results) in hparam_tasks_results.items():

    tradeoff_task = ydnpd.PrivacyUtilityTradeoffTask(hparam_task)
    
    for dev_name, test_name in ydnpd.config.EXPERIMENTS:
        
        display(Markdown(f"## {synth_name}: {dev_name} × {test_name}"))

        tradeoff_task.plot(hparam_results,
                           dev_name=dev_name,
                           test_name=test_name,
                           metric="thresholded_marginals_up_to_3_max_abs_diff_errors")